<a href="https://colab.research.google.com/github/safaltasaxena/ml-learning-mini-projects/blob/main/Naive-Bayes-EmailSpam-CountVectorizer/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

naive bayes

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset.csv


In [ ]:
import pandas as pd
df=pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#dropping those var which have no effect on survival
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [ ]:
target=df.Survived
inputs=df.drop('Survived',axis='columns')

In [ ]:
#gotta change male female to nums as model takes only numeric data
dummies = pd.get_dummies(inputs.Sex).astype(int)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [ ]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [ ]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [ ]:
#now to search if data is missing NA values u do this
inputs.columns[inputs.isna().any()]
#so it says that age coloumn does have NA values

Index(['Age'], dtype='object')

In [ ]:
inputs.Age[:6]

,Age
0,22.000000
1,38.000000
2,26.000000
3,35.000000
4,35.000000
5,29.699118


In [ ]:
#filling up the NA values from mean
inputs.Age=inputs.Age.fillna(inputs.Age.mean())
inputs.Age[:6]

,Age
0,22.000000
1,38.000000
2,26.000000
3,35.000000
4,35.000000
5,29.699118


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2)
#when data distirbuiton is normal we use gaussiannb,visual representation of normal data distirbuition.
#       ^
#       |               .-'''-.
#       |             .'       '.
#       |           .'           '.
#       |         .'               '.
#       |--------'-------------------'----> x (values)
#             (mean - std)     (mean + std)
#
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB()

In [ ]:
model.score(x_test,y_test)

0.8044692737430168

In [ ]:
y_test[:10]

,Survived
45,0
111,0
717,1
600,1
679,1
102,0
737,1
529,0
640,0
406,0


In [ ]:
model.predict(x_test[:10])
#now we can compare our predictions vs actual values

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [ ]:
model.predict_proba(x_test[:10])

array([[9.58265755e-01, 4.17342447e-02],
       [3.75252760e-01, 6.24747240e-01],
       [2.51927472e-01, 7.48072528e-01],
       [2.29774315e-01, 7.70225685e-01],
       [3.35955087e-34, 1.00000000e+00],
       [3.55742784e-01, 6.44257216e-01],
       [3.35033255e-34, 1.00000000e+00],
       [9.01082171e-01, 9.89178291e-02],
       [9.51822980e-01, 4.81770196e-02],
       [9.55572582e-01, 4.44274184e-02]])

building spam email detector

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving spam.csv to spam.csv


In [ ]:
import pandas as pd
df=pd.read_csv("spam.csv")
df.head()
#ham means that mail which is not spam.

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [ ]:
df['spam']=df['Category'].apply(lambda x:1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.Message,df.spam,test_size=0.25)
#now to covert the message coloumn into numeric data we will use new technique

Count vectorizer technique

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
x_train_count=v.fit_transform(x_train)
x_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_count,y_train)

MultinomialNB()

In [ ]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count=v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [ ]:
x_test_count=v.transform(x_test)
model.score(x_test_count,y_test)

0.9856424982053122

rn we are first transforming our raw data then we are feeding it into the model so to avoid all that we dirctly use sklearns's pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
                ('vectorizer',CountVectorizer()),
                ('nb',MultinomialNB())
])

we are writing the same code using a simpler api

In [ ]:
clf.fit(x_train,y_train)
#now we are directly feeding x_train rather x_train_count
#cz now pipeline will internally convert the text into vector first then run it

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [ ]:
clf.score(x_test,y_test)

0.9856424982053122

In [ ]:
clf.predict(emails)

array([0, 1])